### 1. import packages

In [1]:
# import packages
import os
from torch.autograd import Variable as V
from scipy.misc import imresize
import numpy as np
import torch
import numpy

from functools import partial
import re
import random
import signal
import csv
from collections import OrderedDict
from scipy.misc import imread
from multiprocessing import Pool, cpu_count
from multiprocessing.pool import ThreadPool
from scipy.ndimage.interpolation import zoom

import torchvision
import time

### 2. set global varibles

In [2]:
# Settings

######### global settings  #########
GPU = True                                  # running on GPU is highly suggested
TEST_MODE = False                           # turning on the testmode means the code will run on a small dataset.
CLEAN = True                               # set to "True" if you want to clean the temporary large files after generating result
MODEL = 'resnet18'                          # model arch: resnet18, alexnet, resnet50, densenet161
DATASET = 'places365'                       # model trained on: places365 or imagenet
QUANTILE = 0.005                            # the threshold used for activation
SEG_THRESHOLD = 0.04                        # the threshold used for visualization
SCORE_THRESHOLD = 0.04                      # the threshold used for IoU score (in HTML file)
TOPN = 10                                   # to show top N image with highest activation for each unit
PARALLEL = 1                                # how many process is used for tallying (Experiments show that 1 is the fastest)
CATAGORIES = ["object", "part","scene","texture","color"] # concept categories that are chosen to detect: "object", "part", "scene", "material", "texture", "color"
OUTPUT_FOLDER = "../../../project2/bermanm/netdissect/dissection/" + MODEL + "_" + DATASET # result will be stored in this folder

### 3. set some model-dependent variables

In [3]:
########### sub settings ###########

# set data set and image size
if MODEL != 'alexnet': # if the model is not alexnet, use broden1_224
    DATA_DIRECTORY = '../../../project/rcc/deep_learning_hack/netdissect/broden1_224'
    IMG_SIZE = 224
else: # otherwise use broden1_227
    DATA_DIRECTORY = './data/broden1_227'
    IMG_SIZE = 227

# set number of classes
if DATASET == 'places365':
    NUM_CLASSES = 365
elif DATASET == 'imagenet':
    NUM_CLASSES = 1000
    
if MODEL == 'resnet18':
    FEATURE_NAMES = ['layer4']
    if DATASET == 'places365':
        MODEL_FILE = '../../../project2/bermanm/netdissect/ENL_NetDissect/resnet18_places365.pth.tar' # model data is stored here
        MODEL_PARALLEL = True
    elif DATASET == 'imagenet':
        MODEL_FILE = None
        MODEL_PARALLEL = False
elif MODEL == 'densenet161': # cannot run it now
    FEATURE_NAMES = ['features']
    if DATASET == 'places365':
        MODEL_FILE = 'zoo/whole_densenet161_places365_python36.pth.tar'
        MODEL_PARALLEL = False
elif MODEL == 'resnet50': # cannot run it now
    FEATURE_NAMES = ['layer4']
    if DATASET == 'places365':
        MODEL_FILE = 'zoo/whole_resnet50_places365_python36.pth.tar'
        MODEL_PARALLEL = False

# set parameters depending on whether it is test mod
if TEST_MODE:
    WORKERS = 1
    BATCH_SIZE = 4
    TALLY_BATCH_SIZE = 2
    TALLY_AHEAD = 1
    INDEX_FILE = 'index_sm.csv' # what is this file?
    OUTPUT_FOLDER += "_test"
else:
    WORKERS = 12
    BATCH_SIZE = 128
    TALLY_BATCH_SIZE = 16
    TALLY_AHEAD = 4
    INDEX_FILE = 'index.csv'

### 4. helper - decode_index_dict()

In [4]:
def decode_index_dict(row):
    result = {}
    for key, val in row.items():
        if key in ['image', 'split']:
            result[key] = val
        elif key in ['sw', 'sh', 'iw', 'ih']:
            result[key] = int(val)
        else:
            item = [s for s in val.split(';') if s]
            for i, v in enumerate(item):
                if re.match('^\d+$', v):
                    item[i] = int(v)
            result[key] = item
    return result

### 5. helper -  decode_label_dict() 

In [5]:
def decode_label_dict(row):
    result = {}
    for key, val in row.items():
        if key == 'category':
            result[key] = dict((c, int(n))
                for c, n in [re.match('^([^(]*)\(([^)]*)\)$', f).groups()
                    for f in val.split(';')])
        elif key == 'name':
            result[key] = val
        elif key == 'syns':
            result[key] = val.split(';')
        elif re.match('^\d+$', val):
            result[key] = int(val)
        elif re.match('^\d+\.\d*$', val):
            result[key] = float(val)
        else:
            result[key] = val
    return result

### 6. helper - index_has_all_data()

In [6]:
def index_has_all_data(row, categories):
    for c in categories:
        cat_has = False
        for data in row[c]:
            if data:
                cat_has = True
                break
        if not cat_has:
            return False
    return True


### 7. helper - index_has_any_data()

In [7]:
def index_has_any_data(row, categories):
    for c in categories:
        for data in row[c]:
            if data: return True
    return False

### 8. helper - build_numpy_category_map()

In [8]:
def build_numpy_category_map(map_data, key1='code', key2='number'):
    '''
    Input: set of rows with 'number' fields (or another field name key).
    Output: array such that a[number] = the row with the given number.
    '''
    results = list(numpy.zeros((max([d[key] for d in map_data]) + 1),
            dtype=numpy.int16) for key in (key1, key2))
    for d in map_data:
        results[0][d[key1]] = d[key2]
        results[1][d[key2]] = d[key1]
    return results

### 9. load AbstractSegmentation class
    HACK IT LATER

In [9]:
class AbstractSegmentation:
    def all_names(self, category, j):
        raise NotImplementedError
    def size(self, split=None):
        return 0
    def filename(self, i):
        raise NotImplementedError
    def metadata(self, i):
        return self.filename(i)
    @classmethod
    def resolve_segmentation(cls, m):
        return {}

    def name(self, category, i):
        '''
        Default implemtnation for segmentation_data,
        utilizing all_names.
        '''
        all_names = self.all_names(category, i)
        return all_names[0] if len(all_names) else ''

    def segmentation_data(self, category, i, c=0, full=False):
        '''
        Default implemtnation for segmentation_data,
        utilizing metadata and resolve_segmentation.
        '''
        segs = self.resolve_segmentation(
                self.metadata(i), categories=[category])
        if category not in segs:
            return 0
        data = segs[category]
        if not full and len(data.shape) >= 3:
            return data[0]
        return data


### 11. Load SegmentationData Class
    HACK IT LATER

In [11]:
class SegmentationData(AbstractSegmentation):
    '''
    Represents and loads a multi-channel segmentation represented with
    a series of csv files: index.csv lists the images together with
    any label data avilable in each category; category.csv lists
    the categories of segmentations available; and label.csv lists the
    numbers used to describe each label class. In addition, the categories
    each have a separate c_*.csv file describing a dense coding of labels.
    '''

    def __init__(self, directory, categories=None, require_all=False):
        directory = os.path.expanduser(directory)
        self.directory = directory
        with open(os.path.join(directory, INDEX_FILE)) as f:
            self.image = [decode_index_dict(r) for r in csv.DictReader(f)]
        with open(os.path.join(directory, 'category.csv')) as f:
            self.category = OrderedDict()
            for row in csv.DictReader(f):
                if categories and row['name'] in categories:
                    self.category[row['name']] = row
        categories = self.category.keys()
        with open(os.path.join(directory, 'label.csv')) as f:
            label_data = [decode_label_dict(r) for r in csv.DictReader(f)]
        self.label = build_dense_label_array(label_data)
        # Filter out images with insufficient data
        filter_fn = partial(
                index_has_all_data if require_all else index_has_any_data,
                categories=categories)
        self.image = [row for row in self.image if filter_fn(row)]
        # Build dense remapping arrays for labels, so that you can
        # get dense ranges of labels for each category.
        self.category_map = {}
        self.category_unmap = {}
        self.category_label = {}
        for cat in self.category:
            with open(os.path.join(directory, 'c_%s.csv' % cat)) as f:
                c_data = [decode_label_dict(r) for r in csv.DictReader(f)]
            self.category_unmap[cat], self.category_map[cat] = (
                    build_numpy_category_map(c_data))
            self.category_label[cat] = build_dense_label_array(
                    c_data, key='code')

        self.labelcat = self.onehot(self.primary_categories_per_index())

    def primary_categories_per_index(ds):
        '''
        Returns an array of primary category numbers for each label, where the
        first category listed in ds.category_names is given category number 0.
        '''
        catmap = {}
        categories = ds.category_names()
        for cat in categories:
            imap = ds.category_index_map(cat)
            if len(imap) < ds.label_size(None):
                imap = np.concatenate((imap, np.zeros(
                    ds.label_size(None) - len(imap), dtype=imap.dtype)))
            catmap[cat] = imap
        result = []
        for i in range(ds.label_size(None)):
            maxcov, maxcat = max(
                (ds.coverage(cat, catmap[cat][i]) if catmap[cat][i] else 0, ic)
                for ic, cat in enumerate(categories))
            result.append(maxcat)
        return np.array(result)

    def onehot(self, arr, minlength=None):
        '''
        Expands an array of integers in one-hot encoding by adding a new last
        dimension, leaving zeros everywhere except for the nth dimension, where
        the original array contained the integer n.  The minlength parameter is
        used to indcate the minimum size of the new dimension.
        '''
        length = np.amax(arr) + 1
        if minlength is not None:
            length = max(minlength, length)
        result = np.zeros(arr.shape + (length,))
        result[list(np.indices(arr.shape)) + [arr]] = 1
        return result


    def all_names(self, category, j):
        '''All English synonyms for the given label'''
        if category is not None:
            j = self.category_unmap[category][j]
        return [self.label[j]['name']] + self.label[j]['syns']

    def size(self, split=None):
        '''The number of images in this data set.'''
        if split is None:
            return len(self.image)
        return len([im for im in self.image if im['split'] == split])

    def filename(self, i):
        '''The filename of the ith jpeg (original image).'''
        return os.path.join(self.directory, 'images', self.image[i]['image'])

    def split(self, i):
        '''Which split contains item i.'''
        return self.image[i]['split']

    def metadata(self, i):
        '''Extract metadata for image i, For efficient data loading.'''
        return self.directory, self.image[i]

    meta_categories = ['image', 'split', 'ih', 'iw', 'sh', 'sw']

    @classmethod
    def resolve_segmentation(cls, m, categories=None):
        '''
        Resolves a full segmentation, potentially in a differenct process,
        for efficient multiprocess data loading.
        '''
        directory, row = m
        result = {}
        for cat, d in row.items():
            if cat in cls.meta_categories:
                continue
            if not wants(cat, categories):
                continue
            if all(isinstance(data, int) for data in d):
                result[cat] = d
                continue
            out = numpy.empty((len(d), row['sh'], row['sw']), dtype=numpy.int16)
            for i, channel in enumerate(d):
                if isinstance(channel, int):
                    out[i] = channel
                else:
                    rgb = imread(os.path.join(directory, 'images', channel))
                    out[i] = rgb[:,:,0] + rgb[:,:,1] * 256
            result[cat] = out
        return result, (row['sh'], row['sw'])

    def label_size(self, category=None):
        '''
        Returns the number of distinct labels (plus zero), i.e., one
        more than the maximum label number.  If a category is specified,
        returns the number of distinct labels within that category.
        '''
        if category is None:
            return len(self.label)
        else:
            return len(self.category_unmap[category])

    def name(self, category, j):
        '''
        Returns an English name for the jth label.  If a category is
        specified, returns the name for the category-specific nubmer j.
        If category=None, then treats j as a fully unified index number.
        '''
        if category is not None:
            j = self.category_unmap[category][j]
        return self.label[j]['name']

    def frequency(self, category, j):
        '''
        Returns the number of images for which the label appears.
        '''
        if category is not None:
            return self.category_label[category][j]['frequency']
        return self.label[j]['frequency']

    def coverage(self, category, j):
        '''
        Returns the pixel coverage of the label in units of whole-images.
        '''
        if category is not None:
            return self.category_label[category][j]['coverage']
        return self.label[j]['coverage']

    def category_names(self):
        '''
        Returns the set of category names.
        '''
        return list(self.category.keys())

    def category_frequency(self, category):
        '''
        Returns the number of images touched by a category.
        '''
        return float(self.category[category]['frequency'])

    def primary_categories_per_index(self, categories=None):
        '''
        Returns an array of primary category numbers for each label, where
        catagories are indexed according to the list of categories passed,
        or self.category_names() if none.
        '''
        if categories is None:
            categories = self.category_names()
        # Make lists which are nonzero for labels in a category
        catmap = {}
        for cat in categories:
            imap = self.category_index_map(cat)
            if len(imap) < self.label_size(None):
                imap = numpy.concatenate((imap, numpy.zeros(
                    self.label_size(None) - len(imap), dtype=imap.dtype)))
            catmap[cat] = imap
        # For each label, find the category with maximum coverage.
        result = []
        for i in range(self.label_size(None)):
            maxcov, maxcat = max(
                    (self.coverage(cat, catmap[cat][i])
                        if catmap[cat][i] else 0, ic)
                    for ic, cat in enumerate(categories))
            result.append(maxcat)
        # Return the max-coverage cateogry for each label.
        return numpy.array(result)
    def segmentation_data(self, category, i, c=0, full=False, out=None):
        '''
        Returns a 2-d numpy matrix with segmentation data for the ith image,
        restricted to the given category.  By default, maps all label numbers
        to the category-specific dense mapping described in the c_*.csv
        listing; but can be asked to expose the fully unique indexing by
        using full=True.
        '''
        row = self.image[i]
        data_channels = row.get(category, ())
        if c >= len(data_channels):
            channel = 0  # Deal with unlabeled data in this category
        else:
            channel = data_channels[c]
        if out is None:
            out = numpy.empty((row['sh'], row['sw']), dtype=numpy.int16)
        if isinstance(channel, int):
            if not full:
                channel = self.category_map[category][channel]
            out[:,:] = channel  # Single-label for the whole image
            return out
        png = imread(os.path.join(self.directory, 'images', channel))
        if full:
            # Full case: just combine png channels.
            out[...] = png[:,:,0] + png[:,:,1] * 256
        else:
            # Dense case: combine png channels and apply the category map.
            catmap = self.category_map[category]
            out[...] = catmap[png[:,:,0] + png[:,:,1] * 256]
        return out
    
    def full_segmentation_data(self, i,
            categories=None, max_depth=None, out=None):
        '''
        Returns a 3-d numpy tensor with segmentation data for the ith image,
        with multiple layers represnting multiple lables for each pixel.
        The depth is variable depending on available data but can be
        limited to max_depth.
        '''
        row = self.image[i]
        if categories:
            groups = [d for cat, d in row.items() if cat in categories and d]
        else:
            groups = [d for cat, d in row.items() if d and (
                cat not in self.meta_categories)]
        depth = sum(len(c) for c in groups)
        if max_depth is not None:
            depth = min(depth, max_depth)
        # Allocate an array if not already allocated.
        if out is None:
            out = numpy.empty((depth, row['sh'], row['sw']), dtype=numpy.int16)
        i = 0
        # Stack up the result segmentation one channel at a time
        for group in groups:
            for channel in group:
                if isinstance(channel, int):
                    out[i] = channel
                else:
                    png = imread(
                            os.path.join(self.directory, 'images', channel))
                    out[i] = png[:,:,0] + png[:,:,1] * 256
                i += 1
                if i == depth:
                    return out
        # Return above when we get up to depth
        assert False

    def category_index_map(self, category):
        return numpy.array(self.category_map[category])

### 12. helper - setup_signit()

In [12]:
def setup_sigint():
    import threading
    if not isinstance(threading.current_thread(), threading._MainThread):
        return None
    return signal.signal(signal.SIGINT, signal.SIG_IGN)

### 13. helper - restore_sigint()

In [13]:
def restore_sigint(original):
    import threading
    if not isinstance(threading.current_thread(), threading._MainThread):
        return
    if original is None:
        original = signal.SIG_DFL
    signal.signal(signal.SIGINT, original)

### 14. load SegmentationPrefetcher class
    HACK IT LATER

In [14]:
class SegmentationPrefetcher:
    '''
    SegmentationPrefetcher will prefetch a bunch of segmentation
    images using a multiprocessing pool, so you do not have to wait
    around while the files get opened and decoded.  Just request
    batches of images and segmentations calling fetch_batch().
    '''
    def __init__(self, segmentation, split=None, randomize=False,
            segmentation_shape=None, categories=None, once=False,
            start=None, end=None, batch_size=4, ahead=4, thread=False):
        '''
        Constructor arguments:
        segmentation: The AbstractSegmentation to load.
        split: None for no filtering, or 'train' or 'val' etc.
        randomize: True to randomly shuffle order, or a random seed.
        categories: a list of categories to include in each batch.
        batch_size: number of data items for each batch.
        ahead: the number of data items to prefetch ahead.
        '''
        self.segmentation = segmentation
        self.split = split
        self.randomize = randomize
        self.random = random.Random()
        if randomize is not True:
            self.random.seed(randomize)
        self.categories = categories
        self.once = once
        self.batch_size = batch_size
        self.ahead = ahead
        # Initialize the multiprocessing pool
        n_procs = cpu_count()
        if thread:
            self.pool = ThreadPool(processes=n_procs)
        else:
            original_sigint_handler = setup_sigint()
            self.pool = Pool(processes=n_procs, initializer=setup_sigint)
            restore_sigint(original_sigint_handler)
        # Prefilter the image indexes of interest
        if start is None:
            start = 0
        if end is None:
            end = segmentation.size()
        self.indexes = range(start, end)
        if split:
            self.indexes = [i for i in self.indexes
                    if segmentation.split(i) == split]
        if self.randomize:
            self.random.shuffle(self.indexes)
        self.index = 0
        self.result_queue = []
        self.segmentation_shape = segmentation_shape
        # Get dense catmaps
        self.catmaps = [
                segmentation.category_index_map(cat) if cat != 'image' else None
                for cat in categories]

    def next_job(self):
        if self.index < 0:
            return None
        j = self.indexes[self.index]
        result = (j,
                self.segmentation.__class__,
                self.segmentation.metadata(j),
                self.segmentation.filename(j),
                self.categories,
                self.segmentation_shape)
        self.index += 1
        if self.index >= len(self.indexes):
            if self.once:
                self.index = -1
            else:
                self.index = 0
                if self.randomize:
                    # Reshuffle every time through
                    self.random.shuffle(self.indexes)
        return result

    def batches(self):
        '''Iterator for all batches'''
        while True:
            batch = self.fetch_batch()
            if batch is None:
                raise StopIteration
            yield batch

    def fetch_batch(self):
        '''Returns a single batch as an array of dictionaries.'''
        try:
            self.refill_tasks()
            if len(self.result_queue) == 0:
                return None
            result = self.result_queue.pop(0)
            return result.get(31536000)
        except KeyboardInterrupt:
            print("Caught KeyboardInterrupt, terminating workers")
            self.pool.terminate()
            raise

    def fetch_tensor_batch(self, bgr_mean=None, global_labels=False):
        '''Iterator for batches as arrays of tensors.'''
        batch = self.fetch_batch()
        return self.form_caffe_tensors(batch, bgr_mean, global_labels)

    def tensor_batches(self, bgr_mean=None, global_labels=False):
        '''Returns a single batch as an array of tensors, one per category.'''
        while True:
            batch = self.fetch_tensor_batch(
                    bgr_mean=bgr_mean, global_labels=global_labels)
            if batch is None:
                raise StopIteration
            yield batch

    def form_caffe_tensors(self, batch, bgr_mean=None, global_labels=False):
        # Assemble a batch in [{'cat': data,..},..] format into
        # an array of batch tensors, the first for the image, and the
        # remaining for each category in self.categories, in order.
        # This also applies a random flip if needed
        if batch is None:
            return None
        batches = [[] for c in self.categories]
        for record in batch:
            default_shape = (1, record['sh'], record['sw'])
            for c, cat in enumerate(self.categories):
                if cat == 'image':
                    # Normalize image with right RGB order and mean
                    batches[c].append(normalize_image(
                        record[cat], bgr_mean))
                elif global_labels:
                    batches[c].append(normalize_label(
                        record[cat], default_shape, flatten=True))
                else:
                    catmap = self.catmaps[c]
                    batches[c].append(catmap[normalize_label(
                        record[cat], default_shape, flatten=True)])
        return [numpy.concatenate(tuple(m[numpy.newaxis] for m in b))
                for b in batches]

    def refill_tasks(self):
        # It will call the sequencer to ask for a sequence
        # of batch_size jobs (indexes with categories)
        # Then it will call pool.map_async
        while len(self.result_queue) < self.ahead:
            data = []
            while len(data) < self.batch_size:
                job = self.next_job()
                if job is None:
                    break
                data.append(job)
            if len(data) == 0:
                return
            self.result_queue.append(self.pool.map_async(prefetch_worker, data))

    def close(self):
        while len(self.result_queue):
            result = self.result_queue.pop(0)
            if result is not None:
                result.wait(0.001)
        self.pool.close()
        self.poool.cancel_join_thread()

### 15. helper - build_dense_label_array()

In [15]:
def build_dense_label_array(label_data, key='number', allow_none=False):
    '''
    Input: set of rows with 'number' fields (or another field name key).
    Output: array such that a[number] = the row with the given number.
    '''
    result = [None] * (max([d[key] for d in label_data]) + 1)
    for d in label_data:
        result[d[key]] = d
    # Fill in none
    if not allow_none:
        example = label_data[0]
        def make_empty(k):
            return dict((c, k if c is key else type(v)())
                    for c, v in example.items())
        for i, d in enumerate(result):
            if d is None:
                result[i] = dict(make_empty(i))
    return result

### 16. helper - loadmodel()
    HACK IT LATER

In [16]:
def loadmodel(hook_fn):
    if MODEL_FILE is None:
        model = torchvision.models.__dict__[MODEL](pretrained=True)
    else:
        checkpoint = torch.load(MODEL_FILE)
        if type(checkpoint).__name__ == 'OrderedDict' or type(checkpoint).__name__ == 'dict':
            model = torchvision.models.__dict__[MODEL](num_classes=NUM_CLASSES)
            if MODEL_PARALLEL:
                state_dict = {str.replace(k, 'module.', ''): v for k, v in checkpoint[
                    'state_dict'].items()}  # the data parallel layer will add 'module' before each layer name
            else:
                state_dict = checkpoint
            model.load_state_dict(state_dict)
        else:
            model = checkpoint
    for name in FEATURE_NAMES:
        model._modules.get(name).register_forward_hook(hook_fn)
    if GPU:
        model.cuda()
    model.eval()
    return model

### 17. helper - prefetch_worker()

In [17]:
def prefetch_worker(d):
    if d is None:
        return None
    j, typ, m, fn, categories, segmentation_shape = d
    segs, shape = typ.resolve_segmentation(m, categories=categories)
    if segmentation_shape is not None:
        for k, v in segs.items():
            segs[k] = scale_segmentation(v, segmentation_shape)
        shape = segmentation_shape
    # Some additional metadata to provide
    segs['sh'], segs['sw'] = shape
    segs['i'] = j
    segs['fn'] = fn
    if categories is None or 'image' in categories:
        segs['image'] = imread(fn)
    return segs

### 18. helper - scale_segmentation

In [18]:
def scale_segmentation(segmentation, dims, crop=False):
    '''
    Zooms a 2d or 3d segmentation to the given dims, using nearest neighbor.
    '''
    shape = numpy.shape(segmentation)
    if len(shape) < 2 or shape[-2:] == dims:
        return segmentation
    peel = (len(shape) == 2)
    if peel:
        segmentation = segmentation[numpy.newaxis]
    levels = segmentation.shape[0]
    result = numpy.zeros((levels, ) + dims,
            dtype=segmentation.dtype)
    ratio = (1,) + tuple(res / float(orig)
            for res, orig in zip(result.shape[1:], segmentation.shape[1:]))
    if not crop:
        safezoom(segmentation, ratio, output=result, order=0)
    else:
        ratio = max(ratio[1:])
        height = int(round(dims[0] / ratio))
        hmargin = (segmentation.shape[0] - height) // 2
        width = int(round(dims[1] / ratio))
        wmargin = (segmentation.shape[1] - height) // 2
        safezoom(segmentation[:, hmargin:hmargin+height,
            wmargin:wmargin+width],
            (1, ratio, ratio), output=result, order=0)
    if peel:
        result = result[0]
    return result

### 19. helper - safezoom()

In [19]:
def safezoom(array, ratio, output=None, order=0):
    '''Like numpy.zoom, but does not crash when the first dimension
    of the array is of size 1, as happens often with segmentations'''
    dtype = array.dtype
    if array.dtype == numpy.float16:
        array = array.astype(numpy.float32)
    if array.shape[0] == 1:
        if output is not None:
            output = output[0,...]
        result = zoom(array[0,...], ratio[1:],
                output=output, order=order)
        if output is None:
            output = result[numpy.newaxis]
    else:
        result = zoom(array, ratio, output=output, order=order)
        if output is None:
            output = result
    return output.astype(dtype)

### 20. helper - wants()

In [20]:
def wants(what, option):
    if option is None:
        return True
    return what in option

### 21. helper - normalize_image()

In [21]:
def normalize_image(rgb_image, bgr_mean):
    """
    Load input image and preprocess for Caffe:
    - cast to float
    - switch channels RGB -> BGR
    - subtract mean
    - transpose to channel x height x width order
    """
    img = numpy.array(rgb_image, dtype=numpy.float32)
    if (img.ndim == 2):
        img = numpy.repeat(img[:,:,None], 3, axis = 2)
    img = img[:,:,::-1]
    if bgr_mean is not None:
        img -= bgr_mean
    img = img.transpose((2,0,1))
    return img

### 22. helper - normalize_label()

In [22]:
def normalize_label(label_data, shape, flatten=False):
    """
    Given a 0, 1, 2, or 3-dimensional label_data and a default
    shape of the form (1, y, x), returns a 3d tensor by 
    """
    dims = len(numpy.shape(label_data))
    if dims <= 2:
        # Scalar data on this channel: fill shape
        if dims == 1:
            if flatten:
                label_data = label_data[0] if len(label_data) else 0
            else:
                return (numpy.ones(shape, dtype=numpy.int16) *
                        numpy.asarray(label_data, dtype=numpy.int16)
                            [:, numpy.newaxis, numpy.newaxis])
        return numpy.full(shape, label_data, dtype=numpy.int16)
    else:
        if dims == 3:
            if flatten:
                label_data = label_data[0]
            else:
                return label_data
        return label_data[numpy.newaxis]

### 23. Hook_feature()

In [23]:
# what is this?
features_blobs = []

# update feature_blobs
def hook_feature(module, input, output):
    features_blobs.append(output.data.cpu().numpy())

### 24. Load QuantileVector Class
    Based on the optimal KLL quantile algorithm by Karnin, Lang, and Liberty
    from FOCS 2016.  http://ieee-focs.org/FOCS-2016-Papers/3933a071.pdf
    
    HACK IT LATER

In [24]:
# compute quantiles using less memory
class QuantileVector:
    def __init__(self, depth=1, resolution=24 * 1024, buffersize=None,
            dtype=None, seed=None):
        self.resolution = resolution
        self.depth = depth
        # Default buffersize: 128 samples (and smaller than resolution).
        if buffersize is None:
            buffersize = min(128, (resolution + 7) // 8)
        self.buffersize = buffersize
        self.samplerate = 1.0
        self.data = [numpy.zeros(shape=(depth, resolution), dtype=dtype)]
        self.firstfree = [0]
        self.random = numpy.random.RandomState(seed)
        self.extremes = numpy.empty(shape=(depth, 2), dtype=dtype)
        self.extremes.fill(numpy.NaN)
        self.size = 0
    
    def add(self, incoming):
        assert len(incoming.shape) == 2
        assert incoming.shape[1] == self.depth
        self.size += incoming.shape[0]
        # Convert to a flat numpy array.
        if self.samplerate >= 1.0:
            self._add_every(incoming)
            return
        # If we are sampling, then subsample a large chunk at a time.
        self._scan_extremes(incoming)
        chunksize = numpy.ceil[self.buffersize / self.samplerate]
        for index in range(0, len(incoming), chunksize):
            batch = incoming[index:index+chunksize]
            sample = batch[self.random.binomial(1, self.samplerate, len(batch))]
            self._add_every(sample)
        
    def _add_every(self, incoming):
        supplied = len(incoming)
        index = 0
        while index < supplied:
            ff = self.firstfree[0]
            available = self.data[0].shape[1] - ff
            if available == 0:
                if not self._shift():
                    # If we shifted by subsampling, then subsample.
                    incoming = incoming[index:]
                    if self.samplerate >= 0.5:
                        print('SAMPLING')
                        self._scan_extremes(incoming)
                    incoming = incoming[self.random.binomial(1, 0.5,
                        len(incoming - index))]
                    index = 0
                    supplied = len(incoming)
                ff = self.firstfree[0]
                available = self.data[0].shape[1] - ff
            copycount = min(available, supplied - index)
            self.data[0][:,ff:ff + copycount] = numpy.transpose(
                    incoming[index:index + copycount,:])
            self.firstfree[0] += copycount
            index += copycount

    def _shift(self):
        index = 0
        # If remaining space at the current layer is less than half prev
        # buffer size (rounding up), then we need to shift it up to ensure
        # enough space for future shifting.
        while self.data[index].shape[1] - self.firstfree[index] < (
                -(-self.data[index-1].shape[1] // 2) if index else 1):
            if index + 1 >= len(self.data):
                return self._expand()
            data = self.data[index][:,0:self.firstfree[index]]
            data.sort()
            if index == 0 and self.samplerate >= 1.0:
                self._update_extremes(data[:,0], data[:,-1])
            offset = self.random.binomial(1, 0.5)
            position = self.firstfree[index + 1]
            subset = data[:,offset::2]
            self.data[index + 1][:,position:position + subset.shape[1]] = subset
            self.firstfree[index] = 0
            self.firstfree[index + 1] += subset.shape[1]
            index += 1
        return True

    def _scan_extremes(self, incoming):
        # When sampling, we need to scan every item still to get extremes
        self._update_extremes(
                numpy.nanmin(incoming, axis=0),
                numpy.nanmax(incoming, axis=0))

    def _update_extremes(self, minr, maxr):
        self.extremes[:,0] = numpy.nanmin(
                [self.extremes[:, 0], minr], axis=0)
        self.extremes[:,-1] = numpy.nanmax(
                [self.extremes[:, -1], maxr], axis=0)

    def minmax(self):
        if self.firstfree[0]:
            self._scan_extremes(self.data[0][:,:self.firstfree[0]].transpose())
        return self.extremes.copy()

    def _expand(self):
        cap = self._next_capacity()
        if cap > 0:
            # First, make a new layer of the proper capacity.
            self.data.insert(0, numpy.empty(
                shape=(self.depth, cap), dtype=self.data[-1].dtype))
            self.firstfree.insert(0, 0)
        else:
            # Unless we're so big we are just subsampling.
            assert self.firstfree[0] == 0
            self.samplerate *= 0.5
        for index in range(1, len(self.data)):
            # Scan for existing data that needs to be moved down a level.
            amount = self.firstfree[index]
            if amount == 0:
                continue
            position = self.firstfree[index-1]
            # Move data down if it would leave enough empty space there
            # This is the key invariant: enough empty space to fit half
            # of the previous level's buffer size (rounding up)
            if self.data[index-1].shape[1] - (amount + position) >= (
                    -(-self.data[index-2].shape[1] // 2) if (index-1) else 1):
                self.data[index-1][:,position:position + amount] = (
                        self.data[index][:,:amount])
                self.firstfree[index-1] += amount
                self.firstfree[index] = 0
            else:
                # Scrunch the data if it would not.
                data = self.data[index][:,:amount]
                data.sort()
                if index == 1:
                    self._update_extremes(data[:,0], data[:,-1])
                offset = self.random.binomial(1, 0.5)
                scrunched = data[:,offset::2]
                self.data[index][:,:scrunched.shape[1]] = scrunched
                self.firstfree[index] = scrunched.shape[1]
        return cap > 0

    def _next_capacity(self):
        cap = numpy.ceil(self.resolution * numpy.power(0.67, len(self.data)))
        if cap < 2:
            return 0
        return max(self.buffersize, int(cap))

    def _weighted_summary(self, sort=True):
        if self.firstfree[0]:
            self._scan_extremes(self.data[0][:,:self.firstfree[0]].transpose())
        size = sum(self.firstfree) + 2
        weights = numpy.empty(
            shape=(size), dtype='float32') # floating point
        summary = numpy.empty(
            shape=(self.depth, size), dtype=self.data[-1].dtype)
        weights[0:2] = 0
        summary[:,0:2] = self.extremes
        index = 2
        for level, ff in enumerate(self.firstfree):
            if ff == 0:
                continue
            summary[:,index:index + ff] = self.data[level][:,:ff]
            weights[index:index + ff] = numpy.power(2.0, level)
            index += ff
        assert index == summary.shape[1]
        if sort:
            order = numpy.argsort(summary)
            summary = summary[numpy.arange(self.depth)[:,None], order]
            weights = weights[order]
        return (summary, weights)

    def quantiles(self, quantiles, old_style=False):
        if self.size == 0:
            return numpy.full((self.depth, len(quantiles)), numpy.nan)
        summary, weights = self._weighted_summary()
        cumweights = numpy.cumsum(weights, axis=-1) - weights / 2
        if old_style:
            # To be convenient with numpy.percentile
            cumweights -= cumweights[:,0:1]
            cumweights /= cumweights[:,-1:]
        else:
            cumweights /= numpy.sum(weights, axis=-1, keepdims=True)
        result = numpy.empty(shape=(self.depth, len(quantiles)))
        for d in range(self.depth):
            result[d] = numpy.interp(quantiles, cumweights[d], summary[d])
        return result

    def integrate(self, fun):
        result = None
        for level, ff in enumerate(self.firstfree):
            if ff == 0:
                continue
            term = numpy.sum(
                    fun(self.data[level][:,:ff]) * numpy.power(2.0, level),
                    axis=-1)
            if result is None:
                result = term
            else:
                result += term
        if result is not None:
            result /= self.samplerate
        return result

    def percentiles(self, percentiles):
        return self.quantiles(percentiles, old_style=True)

    def readout(self, count, old_style=True):
        return self.quantiles(
                numpy.linspace(0.0, 1.0, count), old_style=old_style)

### 25. load FeatureOperator class

In [25]:
class FeatureOperator:
    def __init__(self):
            # make a new directory if the give path does not have an output folder
            if not os.path.exists(OUTPUT_FOLDER): 
                os.makedirs(os.path.join(OUTPUT_FOLDER, 'image'))
            # turn data into a SegmentationData class (see previous class definition) 
            self.data = SegmentationData(DATA_DIRECTORY, categories=CATAGORIES)
            # prefetch some segmentation images using a multiprocessing pool
            self.loader = SegmentationPrefetcher(self.data,categories=['image'],once=True,batch_size=BATCH_SIZE)
            self.mean = [109.5388,118.6897,124.6901]
            
    def feature_extraction(self, model=None, memmap=True):
        loader = self.loader
        # extract the max value activaiton for each image
        maxfeatures = [None] * len(FEATURE_NAMES) # len -> how many features? FEATURE_NAMES = ["layer4"]
        wholefeatures = [None] * len(FEATURE_NAMES)
        features_size = [None] * len(FEATURE_NAMES)
        # get path of features_size_file
        features_size_file = os.path.join(OUTPUT_FOLDER, "feature_size.npy")
    
        if memmap:
            skip = True
            # define paths for mmap_files
            mmap_files =  [os.path.join(OUTPUT_FOLDER, "%s.mmap" % feature_name)  for feature_name in FEATURE_NAMES]
            mmap_max_files = [os.path.join(OUTPUT_FOLDER, "%s_max.mmap" % feature_name) for feature_name in FEATURE_NAMES]
            # if feature size file exists already
            if os.path.exists(features_size_file):
                # load the file -> something similar to a numpy dictionary
                features_size = np.load(features_size_file)
            # if feature size file does not exist
            else:
                skip = False
            # note: zip (kind of) combine two iterables into tuples
            for i, (mmap_file, mmap_max_file) in enumerate(zip(mmap_files,mmap_max_files)):
                if os.path.exists(mmap_file) and os.path.exists(mmap_max_file) and features_size[i] is not None:
                    print('loading features %s' % FEATURE_NAMES[i])
                    # np.memmap: create a memory-map to an array stored in a binary file on disk.
                    wholefeatures[i] = np.memmap(mmap_file, dtype=float,mode='r', shape=tuple(features_size[i]))
                    maxfeatures[i] = np.memmap(mmap_max_file, dtype=float, mode='r', shape=tuple(features_size[i][:2]))
                else:
                    print('file missing, loading from scratch')
                    skip = False
        # skip is false if any of the files is missing
            if skip:
                return wholefeatures, maxfeatures
        
        # now run feature extraction no matter memmap is used or not
        # loader.indexes & loader.batch_size comes from prefetcher class 
        # compute number of batches by divide the total number of samples by the batch size
        num_batches = (len(loader.indexes) + loader.batch_size - 1) / loader.batch_size
        # loader.tensor_batches is a batch of an array of tensors
        for batch_idx,batch in enumerate(loader.tensor_batches(bgr_mean=self.mean)):
            # del: remove an item in the list given an index; here empty the who list
            del features_blobs[:]
            input = batch[0]
            batch_size = len(input)
            print('extracting feature from batch %d / %d' % (batch_idx+1, num_batches))
            #reverse the second intex 
            # note the second subarray is reversed
            input = torch.from_numpy(input[:, ::-1, :, :].copy())
            input.div_(255.0 * 0.224) # ?
            # check if GPU is used
            if GPU:
                input = input.cuda()
            # V = autograd.variable
            input_var = V(input,volatile=True)  
            # feed forward into the model?
            logit = model.forward(input_var)
            while np.isnan(logit.data.max()): # ok the coder doesn't know when this scenario will happen
                print("nan") 
                del features_blobs[:]
                logit = model.forward(input_var)
                
            # initialize maxfeature variable
            if maxfeatures[0] is None:
                for i, feat_batch in enumerate(features_blobs):
                    # I don't understand features_blobs
                    size_features = (len(loader.indexes), feat_batch.shape[1])
                    # if memmap, get the max feature
                    if memmap:
                         maxfeatures[i] = np.memmap(mmap_max_files[i],dtype=float,mode='w+',shape=size_features)
                    # otherwise initialize with all zeros
                    else:
                        maxfeatures[i] = np.zeros(size_features)
            
            # what is feat_batch <- what is feature_blobs?
            # initialize wholefeature variable
            if len(feat_batch.shape) == 4 and wholefeatures[0] is None:
                for i, feat_batch in enumerate(features_blobs):
                    size_features = (len(loader.indexes), 
                                     feat_batch.shape[1], 
                                     feat_batch.shape[2], 
                                     feat_batch.shape[3])
                    features_size[i] = size_features
                    if memmap:
                        wholefeatures[i] = np.memmap(mmap_files[i], dtype=float, mode='w+', shape=size_features)
                    else:
                        wholefeatures[i] = np.zeros(size_features)
            
            np.save(features_size_file, features_size)
            # set start index and end index
            start_idx = batch_idx*BATCH_SIZE
            end_idx = min((batch_idx+1)*BATCH_SIZE, len(loader.indexes))
            
            # get feature variable
            for i, feat_batch in enumerate(features_blobs):
                if len(feat_batch.shape) == 4:
                    wholefeatures[i][start_idx:end_idx] = feat_batch
                    maxfeatures[i][start_idx:end_idx] = np.max(np.max(feat_batch,3),2)
                elif len(feat_batch.shape) == 3:
                    maxfeatures[i][start_idx:end_idx] = np.max(feat_batch, 2)
                elif len(feat_batch.shape) == 2:
                    maxfeatures[i][start_idx:end_idx] = feat_batch
        
        if len(feat_batch.shape) == 2:
            wholefeatures = maxfeatures
        return wholefeatures,maxfeatures
    
    def quantile_threshold(self, features, savepath=''):
        # set path for quantile threshold
        qtpath = os.path.join(OUTPUT_FOLDER, savepath)
        # if the word has been done, simply return
        if savepath and os.path.exists(qtpath):
             return np.load(qtpath)
        print("calculating quantile threshold")
        quant = QuantileVector(depth=features.shape[1], seed=1)
        start_time = time.time()
        last_batch_time = start_time
        # setting batch size ???
        batch_size = 64
        for i in range(0, features.shape[0], batch_size):
            # for each batch
            # keep track of the time
            batch_time = time.time()
            rate = i / (batch_time - start_time + 1e-15)
            batch_rate = batch_size / (batch_time - last_batch_time + 1e-15)
            last_batch_time = batch_time
            print('Processing quantile index %d: %f %f' % (i, rate, batch_rate))
            # fetch elements in the batch
            batch = features[i:i + batch_size]
            batch = np.transpose(batch, axes=(0, 2, 3, 1)).reshape(-1, features.shape[1]) # fine
            # Quantile Vector method: adding batch
            quant.add(batch)
        # QUANTILE = threshold for activation; = 0.005 by default
        print(quant.readout(1000))
        print(1-QUANTILE)
        ret = quant.readout(1000)[:, int(1000 * (1-QUANTILE)-1)]
        if savepath:
            np.save(qtpath, ret)
        return ret
    
    def tally_job(args):
        features, data, threshold, tally_labels, tally_units, tally_units_cat, tally_both, start, end = args
        # number of units
        units = features.shape[1]
        size_RF = (IMG_SIZE / features.shape[2], IMG_SIZE / features.shape[3])
        fieldmap = ((0, 0), size_RF, size_RF)
        pd = SegmentationPrefetcher(data, categories=data.category_names(),
                                    once=True, batch_size=TALLY_BATCH_SIZE,
                                    ahead=TALLY_AHEAD, start=start, end=end)
        # keep track of time 
        count = start
        start_time = time.time()
        last_batch_time = start_time
        for batch in pd.batches():
            batch_time = time.time()
            rate = (count - start) / (batch_time - start_time + 1e-15)
            batch_rate = len(batch) / (batch_time - last_batch_time + 1e-15)
            last_batch_time = batch_time
            print('labelprobe image index %d, items per sec %.4f, %.4f' % (count, rate, batch_rate))
            
            for concept_map in batch:
                count += 1
                # get the index of the image
                img_index = concept_map['i']
                scalars, pixels = [], []
                # for each category
                for cat in data.category_names():
                    # get corresponding 
                    label_group = concept_map[cat]
                    shape = np.shape(label_group)
                    if len(shape) % 2 == 0:
                        label_group = [label_group]
                    if len(shape) < 2:
                        scalars += label_group
                    else:
                        pixels.append(label_group)

                    for scalar in scalars:
                        tally_labels[scalar] += concept_map['sh'] * concept_map['sw']
                    if pixels:
                        pixels = np.concatenate(pixels)
                        tally_label = np.bincount(pixels.ravel())
                        if len(tally_label) > 0:
                            tally_label[0] = 0
                        tally_labels[:len(tally_label)] += tally_label

                # for each individual unit
                for unit_id in range(units):
                    feature_map = features[img_index][unit_id]
                    # if max value exceed threshold for the unit
                    if feature_map.max() > threshold[unit_id]:
                        # imresize? 
                        mask = imresize(feature_map, (concept_map['sh'], concept_map['sw']), mode='F')
                        indexes = np.argwhere(mask > threshold[unit_id])

                        tally_units[unit_id] += len(indexes)
                        if len(pixels) > 0:
                            tally_bt = np.bincount(pixels[:, indexes[:, 0], indexes[:, 1]].ravel())
                            if len(tally_bt) > 0:
                                tally_bt[0] = 0
                            tally_cat = np.dot(tally_bt[None,:], data.labelcat[:len(tally_bt), :])[0]
                            tally_both[unit_id,:len(tally_bt)] += tally_bt
                        for scalar in scalars:
                            tally_cat += data.labelcat[scalar]
                            tally_both[unit_id, scalar] += len(indexes)
                        tally_units_cat[unit_id] += len(indexes) * (tally_cat > 0)

                
        
    def tally(self, features, threshold, savepath=''):
        csvpath = os.path.join(OUTPUT_FOLDER, savepath)
        if savepath and os.path.exists(csvpath):
            return load_csv(csvpath)
        
        # initialize 
        units = features.shape[1]
        labels = len(self.data.label)
        categories = self.data.category_names()
        tally_both = np.zeros((units,labels),dtype=np.float64)
        tally_units = np.zeros(units,dtype=np.float64)
        tally_units_cat = np.zeros((units,len(categories)), dtype=np.float64)
        tally_labels = np.zeros(labels,dtype=np.float64)
        
        # OK seems like something for parallel computing; check it later
        if PARALLEL > 1:
            psize = int(np.ceil(float(self.data.size()) / PARALLEL))
            ranges = [(s, min(self.data.size(), s + psize)) for s in range(0, self.data.size(), psize) if
                      s < self.data.size()]
            params = [(features, self.data, threshold, tally_labels, tally_units, tally_units_cat, tally_both) + r for r in ranges]
            threadpool = pool.ThreadPool(processes=PARALLEL)
            threadpool.map(FeatureOperator.tally_job, params)
        else:
            FeatureOperator.tally_job((features, self.data, threshold, tally_labels, tally_units, tally_units_cat, tally_both, 0, self.data.size()))

        primary_categories = self.data.primary_categories_per_index()
        # compute dot product between 
        tally_units_cat = np.dot(tally_units_cat, self.data.labelcat.T)
        # iou intersection over units
        iou = tally_both / (tally_units_cat + tally_labels[np.newaxis,:] - tally_both + 1e-10)
        pciou = np.array([iou * (primary_categories[np.arange(iou.shape[1])] == ci)[np.newaxis, :] for ci in range(len(self.data.category_names()))])
        label_pciou = pciou.argmax(axis=2)
        name_pciou = [
            [self.data.name(None, j) for j in label_pciou[ci]]
            for ci in range(len(label_pciou))]
        score_pciou = pciou[
            np.arange(pciou.shape[0])[:, np.newaxis],
            np.arange(pciou.shape[1])[np.newaxis, :],
            label_pciou]
        bestcat_pciou = score_pciou.argsort(axis=0)[::-1]
        ordering = score_pciou.max(axis=0).argsort()[::-1]
        rets = [None] * len(ordering)
        
        for i,unit in enumerate(ordering):
            bestcat = bestcat_pciou[0, unit]
            data = {
                'unit': (unit + 1),
                'category': categories[bestcat],
                'label': name_pciou[bestcat][unit],
                'score': score_pciou[bestcat][unit]
            }
            for ci, cat in enumerate(categories):
                label = label_pciou[ci][unit]
                data.update({
                    '%s-label' % cat: name_pciou[ci][unit],
                    '%s-truth' % cat: tally_labels[label],
                    '%s-activation' % cat: tally_units_cat[unit, label],
                    '%s-intersect' % cat: tally_both[unit, label],
                    '%s-iou' % cat: score_pciou[ci][unit]
                })
            rets[i] = data

        if savepath:
            import csv
            csv_fields = sum([[
                '%s-label' % cat,
                '%s-truth' % cat,
                '%s-activation' % cat,
                '%s-intersect' % cat,
                '%s-iou' % cat] for cat in categories],
                ['unit', 'category', 'label', 'score'])
            with open(csvpath, 'w') as f:
                writer = csv.DictWriter(f, csv_fields)
                writer.writeheader()
                for i in range(len(ordering)):
                    writer.writerow(rets[i])
        return rets
            
            

### 27. Define a Feature_Operator variable

In [26]:
fo = FeatureOperator()

###  28. Do feature extraction()

In [27]:
model = loadmodel(hook_feature)
features, maxfeature = fo.feature_extraction(model=model)

loading features layer4


#### Check features and maxfeatures

In [28]:
print(len(features))
print(features[0].shape)
#print(maxfeature[0].shape)
#print(features[0][1 : 1 + 64])

1
(63305, 512, 7, 7)


### 29. Calculate threshold

In [29]:
# want to have P(a_k > T_k) = 0.005, where k is a unit
for layer_id,layer in enumerate(FEATURE_NAMES):
    print(layer_id, layer)
    thresholds = fo.quantile_threshold(features[layer_id],savepath="quantile.npy")

0 layer4


### History lost :( study Tally again

In [30]:
layer_id = 0
layer = FEATURE_NAMES[0]

In [31]:
savepath = "tally.csv"
csvpath = os.path.join(OUTPUT_FOLDER, savepath)

In [32]:
if savepath and os.path.exists(csvpath):
    print("tally exist")
    tally_result = load_csv(csvpath)

In [33]:
f = features[layer_id]
print(f)

[[[[0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
    0.00000000e+00 0.00000000e+00]
   [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
    0.00000000e+00 0.00000000e+00]
   [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
    0.00000000e+00 1.31716684e-01]
   ...
   [0.00000000e+00 0.00000000e+00 1.51383251e-01 ... 4.41018790e-01
    1.36240888e+00 1.22327912e+00]
   [1.05288410e+00 1.19605005e+00 1.29995263e+00 ... 1.20202816e+00
    2.18029690e+00 1.83771491e+00]
   [1.84581208e+00 1.43044806e+00 1.54802322e+00 ... 1.28344512e+00
    1.62736440e+00 1.25797927e+00]]

  [[0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
    0.00000000e+00 0.00000000e+00]
   [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
    0.00000000e+00 0.00000000e+00]
   [2.10698515e-01 5.89928776e-02 1.88222807e-02 ... 1.24826215e-01
    0.00000000e+00 0.00000000e+00]
   ...
   [6.16111338e-01 2.52829999e-01 0.00000000e+00 ... 0.00

In [34]:
units = f.shape[1]
print(units)

512


In [35]:
labels = len(fo.data.label)
print(labels)

1198


In [36]:
categories = fo.data.category_names()
print(categories)

['color', 'object', 'part', 'scene', 'texture']


In [37]:
tally_both = np.zeros((units,labels),dtype=np.float64) # 512 x 1198

In [38]:
tally_units = np.zeros(units,dtype=np.float64) # 512

In [39]:
tally_units_cat = np.zeros((units,len(categories)), dtype=np.float64) # 512 x 5

In [40]:
tally_labels = np.zeros(labels,dtype=np.float64) # 1198

### Tally job

In [41]:
threshold = thresholds
d = fo.data
start = 0
end = fo.data.size()

In [42]:
size_RF = (IMG_SIZE / f.shape[2], IMG_SIZE / f.shape[3])
print(size_RF)

(32.0, 32.0)


In [43]:
fieldmap = ((0, 0), size_RF, size_RF)
print(fieldmap)

((0, 0), (32.0, 32.0), (32.0, 32.0))


In [44]:
pd = SegmentationPrefetcher(d, categories=d.category_names(),
                                    once=True, batch_size=TALLY_BATCH_SIZE,
                                    ahead=TALLY_AHEAD, start=start, end=end)

/software/Anaconda3-5.0.1-el7-x86_64/envs/DL_GPU_cuda_9.0/lib/python3.6/site-packages/ipykernel/__main__.py:128: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/software/Anaconda3-5.0.1-el7-x86_64/envs/DL_GPU_cuda_9.0/lib/python3.6/site-packages/ipykernel/__main__.py:128: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/software/Anaconda3-5.0.1-el7-x86_64/envs/DL_GPU_cuda_9.0/lib/python3.6/site-packages/ipykernel/__main__.py:128: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/software/Anaconda3-5.0.1-el7-x86_64/envs/DL_GPU_cuda_9.0/lib/python3.6/site-packages/ipykernel/__main__.py:128: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.


In [45]:
count = start

In [57]:
f[0][2]

memmap([[0.43572149, 0.52332854, 0.59500879, 1.0443548 , 1.25218737,
         1.25133681, 0.91424435],
        [0.16593704, 0.2995311 , 0.61378074, 1.26466084, 1.57409823,
         1.57594156, 1.06530654],
        [0.        , 0.        , 0.        , 0.        , 0.23191556,
         0.60445142, 0.56907916],
        [0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        ]])

In [65]:
d.labelcat[:12, :]

array([[0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.]])

In [70]:
indexes[0]

array([51, 56])

In [74]:
print(tally_bt)

[   0  126   56   47  280  845    7    0    0   86    0    0    0    0
    0    0 1306    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0  126    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0 

In [77]:
tally_cat = np.dot(tally_bt[None,:], d.labelcat[:len(tally_bt), :])

In [79]:
tally_bt[None, :]

array([[   0,  126,   56,   47,  280,  845,    7,    0,    0,   86,    0,
           0,    0,    0,    0,    0, 1306,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
         126,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0, 

In [80]:
d.labelcat[:len(tally_bt), :]

array([[0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.]])

In [46]:
for batch in pd.batches():
    for concept_map in batch:
        print(concept_map)
        count += 1
        img_index = concept_map['i']
        scalars, pixels = [], []
        for cat in d.category_names():
            #print("category: ", cat)
            label_group = concept_map[cat]
            #print("label_group: ", label_group)
            shape = np.shape(label_group)
            #print("shape: ", shape)
            if len(shape) % 2 == 0:    # When will this happen?
                label_group = [label_group]
            if len(shape) < 2:
                scalars += label_group
            else:
                pixels.append(label_group)
            #print("scalars: ", scalars)
            #print("pixels: ", pixels)
        
        for scalar in scalars:
            tally_labels[scalar] += concept_map['sh'] * concept_map['sw'] # what are sh and sw? (12544)
            #print("update scalar", scalar, tally_labels[scalar])
        if pixels:
            pixels = np.concatenate(pixels)
            tally_label = np.bincount(pixels.ravel())  # count the occurence of each color?
            #print("length of tally_label", len(tally_label))
            #print("bincount tally", tally_label)
            if len(tally_label) > 0:
                tally_label[0] = 0   # ignore 0 color -> white or something?
            tally_labels[:len(tally_label)] += tally_label # update color labels? but what if some colors are missed?
            
        for unit_id in range(units):
            feature_map = f[img_index][unit_id]  # 7 x 7
            #print("feature_map:", feature_map)
            if feature_map.max() > threshold[unit_id]: # when there is a feature > threshold for this unit_id
                print("unit:", unit_id)
                # interpolation
                #print("feature map: ", feature_map)
                mask = imresize(feature_map, (concept_map['sh'], concept_map['sw']), mode='F') # F: 32-bit floating point pixels
                #print("first row of mask: ", mask[0])
                indexes = np.argwhere(mask > threshold[unit_id])
                #print("shape of indexes", indexes.shape)
                
                tally_units[unit_id] += len(indexes) # add number of indexes > threshold
                
                if len(pixels) > 0:
                    tally_bt = np.bincount(pixels[:, indexes[:, 0], indexes[:, 1]].ravel()) # find all related colors
                    
                    if len(tally_bt) > 0:
                        tally_bt[0] = 0
                    tally_cat = np.dot(tally_bt[None,:], d.labelcat[:len(tally_bt), :])[0] # get colors count
                    #print("tally_cat", tally_cat)
                    tally_both[unit_id,:len(tally_bt)] += tally_bt # specified to unit_id
                
                for scalar in scalars:
                    tally_cat += data.labelcat[scalar] # add the corresponding category
                    tally_both[unit_id, scalar] += len(indexes)
                tally_units_cat[unit_id] += len(indexes) * (tally_cat > 0)
        break
    break

{'color': array([[[3, 3, 3, ..., 3, 3, 0],
        [3, 3, 3, ..., 3, 3, 0],
        [8, 8, 8, ..., 3, 3, 0],
        ...,
        [2, 2, 2, ..., 1, 4, 0],
        [2, 2, 2, ..., 2, 2, 0],
        [2, 2, 2, ..., 5, 4, 0]]], dtype=int16), 'object': [], 'part': [], 'scene': [], 'texture': [], 'sh': 112, 'sw': 112, 'i': 0, 'fn': '../../../project/rcc/deep_learning_hack/netdissect/broden1_224/images/opensurfaces/25605.jpg'}
unit: 23
unit: 25
unit: 36
unit: 43
unit: 47
unit: 54
unit: 90
unit: 119
unit: 150
unit: 151
unit: 211
unit: 224
unit: 227
unit: 228
unit: 233
unit: 260
unit: 294
unit: 309
unit: 362
unit: 367
unit: 378
unit: 394
unit: 415
unit: 431
unit: 455
unit: 459
unit: 464
unit: 466
unit: 480


/software/Anaconda3-5.0.1-el7-x86_64/envs/DL_GPU_cuda_9.0/lib/python3.6/site-packages/ipykernel/__main__.py:41: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


In [52]:
tally_units_cat[480]

array([944.,   0.,   0.,   0.,   0.])

In [57]:
tally_both[480].shape

(1198,)

In [59]:
color = concept_map["color"]

In [60]:
color

array([[[3, 3, 3, ..., 3, 3, 0],
        [3, 3, 3, ..., 3, 3, 0],
        [8, 8, 8, ..., 3, 3, 0],
        ...,
        [2, 2, 2, ..., 1, 4, 0],
        [2, 2, 2, ..., 2, 2, 0],
        [2, 2, 2, ..., 5, 4, 0]]], dtype=int16)

### Back to Tally

In [63]:
d.primary_categories_per_index() # as its name suggests

array([4, 0, 0, ..., 1, 1, 1])

In [64]:
primary_categories = d.primary_categories_per_index()

In [65]:
tally_units_cat = np.dot(tally_units_cat, d.labelcat.T)

In [66]:
tally_units_cat.shape

(512, 1198)

In [75]:
print(tally_units_cat[480])

[  0. 944. 944. ...   0.   0.   0.]


In [68]:
print(np.newaxis)

None


In [76]:
tally_labels[np.newaxis, :].shape

(1, 1198)

In [77]:
tally_both.shape

(512, 1198)

In [78]:
iou = tally_both / (tally_units_cat + tally_labels[np.newaxis,:] - tally_both + 1e-10)

In [80]:
d = iou[iou > 0]

In [84]:
iou.shape

(512, 1198)

In [91]:
len(d.category_names())

5

In [107]:
temp = np.array([iou * (primary_categories[np.arange(iou.shape[1])] == ci)[np.newaxis, :] for ci in range(len(d.category_names()))])

In [110]:
temp

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

    

In [109]:
primary_categories[np.arange(iou.shape[1])]

array([4, 0, 0, ..., 1, 1, 1])

In [112]:
pciou = np.array([iou * (primary_categories[np.arange(iou.shape[1])] == ci)[np.newaxis, :] for ci in range(len(d.category_names()))])

In [114]:
pciou.shape

(5, 512, 1198)

In [115]:
label_pciou = pciou.argmax(axis=2) # find largest label per each unit & category

In [128]:
pciou[0, 480, :12]

array([0.        , 0.21144556, 0.01672598, 0.04283548, 0.00583942,
       0.0008881 , 0.00191755, 0.        , 0.0319099 , 0.        ,
       0.        , 0.        ])

In [129]:
label_pciou[0, 480]

1

In [131]:
d.name(None, 1)

'black-c'

In [132]:
len(label_pciou)

5

In [133]:
# get label names
name_pciou = [
            [d.name(None, j) for j in label_pciou[ci]]
            for ci in range(len(label_pciou))]

In [136]:
score_pciou = pciou[
            np.arange(pciou.shape[0])[:, np.newaxis],
            np.arange(pciou.shape[1])[np.newaxis, :],
            label_pciou]

In [138]:
score_pciou.shape

(5, 512)

In [145]:
print(score_pciou[0])

[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.102118
 0.         0.21334368 0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.15341747 0.         0.         0.         0.         0.
 0.         0.04573805 0.         0.         0.         0.08308605
 0.         0.         0.         0.         0.         0.
 0.30622471 0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.0509434  0.         0.         0.         0.         0.
 0.         0.         0.         0.      

In [139]:
bestcat_pciou = score_pciou.argsort(axis=0)[::-1]

In [149]:
ordering = score_pciou.max(axis=0).argsort()[::-1]

In [150]:
ordering

array([228, 227, 378,  54,  25, 480, 459, 224,  36, 233, 260, 119, 309,
       294,  23, 151, 367,  47, 394, 464,  90,  43, 431, 466, 455, 211,
       150, 362, 168, 171, 174, 175, 182, 163, 170, 181, 164, 162, 180,
       165, 166, 169, 179, 172, 178, 177, 167, 176, 173, 511, 161, 144,
       132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 145,
       160, 146, 147, 148, 149, 152, 153, 154, 183, 156, 157, 158, 159,
       155, 192, 184, 232, 217, 218, 219, 220, 221, 222, 223, 225, 226,
       229, 230, 231, 234, 215, 235, 236, 237, 238, 239, 240, 241, 242,
       243, 244, 245, 246, 216, 214, 185, 198, 186, 187, 188, 189, 190,
       191, 130, 193, 194, 195, 196, 197, 199, 213, 200, 201, 202, 203,
       204, 205, 206, 207, 208, 209, 210, 212, 131, 123, 129,  32,  34,
        35,  37,  38,  39,  40,  41,  42,  44,  45,  46,  48,  49,  50,
        51,  52,  53,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        33,  31, 128,  30,   1,   2,   3,   4,   5,   6,   7,   

In [51]:
print(tally_labels)

[    0. 13130.  4991. ...     0.     0.     0.]


#### Check thresholds

### 30. Do Tally

In [29]:
#print(fo.data.label)
#print(fo.data.category_names())
print(features.shape)
print(IMG_SIZE)

AttributeError: 'list' object has no attribute 'shape'

In [32]:
#def tally(self, features, threshold, savepath=''):
savepath = "tally.csv"
threshold = thresholds
csvpath = os.path.join(OUTPUT_FOLDER, savepath)
#if savepath and os.path.exists(csvpath):   
    #load_csv(csvpath)
features = features[0]

# initialize 
units = features.shape[1]
labels = len(fo.data.label)
categories = fo.data.category_names()
tally_both = np.zeros((units,labels),dtype=np.float64)
tally_units = np.zeros(units,dtype=np.float64)
tally_units_cat = np.zeros((units,len(categories)), dtype=np.float64)
tally_labels = np.zeros(labels,dtype=np.float64)

data = fo.data
start = 0
end = fo.data.size()
#FeatureOperator.tally_job((features, fp.data, threshold, tally_labels, tally_units, tally_units_cat, tally_both, 0, fo.data.size()))
#features, data, threshold, tally_labels, tally_units, tally_units_cat, tally_both, start, end = args

units = features.shape[1]
size_RF = (IMG_SIZE / features.shape[2], IMG_SIZE / features.shape[3])
fieldmap = ((0, 0), size_RF, size_RF)
    
        

In [33]:
pd = SegmentationPrefetcher(data, categories=data.category_names(),
                                    once=True, batch_size=TALLY_BATCH_SIZE,
                                    ahead=TALLY_AHEAD, start=start, end=end)

/home/kywch/.conda/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:128: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/home/kywch/.conda/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:128: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/home/kywch/.conda/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:128: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/home/kywch/.conda/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:128: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/home/kywch/.conda/envs/fastai/lib/python3.6/site-packages/ipykernel_launche

In [ ]:
count = start
start_time = time.time()
last_batch_time = start_time
cbatch = 0
for batch in pd.batches():
    cbatch += 1
    cconcept = 0
    for concept_map in batch:
        cconcept += 1
        print("cbatch = {}, cconcept = {}, length = {}".format(cbatch, cconcept, len(concept_map)))
        
#batch = pd.batches()
#batch_time = time.time()
#rate = (count - start) / (batch_time - start_time + 1e-15)
#batch_rate = len(batch) / (batch_time - last_batch_time + 1e-15)
#last_batch_time = batch_time
#print('labelprobe image index %d, items per sec %.4f, %.4f' % (count, rate, batch_rate))


cbatch = 1, cconcept = 1, length = 9
cbatch = 1, cconcept = 2, length = 9
cbatch = 1, cconcept = 3, length = 9
cbatch = 1, cconcept = 4, length = 9
cbatch = 1, cconcept = 5, length = 9
cbatch = 1, cconcept = 6, length = 9
cbatch = 1, cconcept = 7, length = 9
cbatch = 1, cconcept = 8, length = 9
cbatch = 1, cconcept = 9, length = 9
cbatch = 1, cconcept = 10, length = 9
cbatch = 1, cconcept = 11, length = 9
cbatch = 1, cconcept = 12, length = 9
cbatch = 1, cconcept = 13, length = 9
cbatch = 1, cconcept = 14, length = 9
cbatch = 1, cconcept = 15, length = 9
cbatch = 1, cconcept = 16, length = 9
cbatch = 2, cconcept = 1, length = 9
cbatch = 2, cconcept = 2, length = 9
cbatch = 2, cconcept = 3, length = 9
cbatch = 2, cconcept = 4, length = 9
cbatch = 2, cconcept = 5, length = 9
cbatch = 2, cconcept = 6, length = 9
cbatch = 2, cconcept = 7, length = 9
cbatch = 2, cconcept = 8, length = 9
cbatch = 2, cconcept = 9, length = 9
cbatch = 2, cconcept = 10, length = 9
cbatch = 2, cconcept = 11, len

cbatch = 15, cconcept = 1, length = 9
cbatch = 15, cconcept = 2, length = 9
cbatch = 15, cconcept = 3, length = 9
cbatch = 15, cconcept = 4, length = 9
cbatch = 15, cconcept = 5, length = 9
cbatch = 15, cconcept = 6, length = 9
cbatch = 15, cconcept = 7, length = 9
cbatch = 15, cconcept = 8, length = 9
cbatch = 15, cconcept = 9, length = 9
cbatch = 15, cconcept = 10, length = 9
cbatch = 15, cconcept = 11, length = 9
cbatch = 15, cconcept = 12, length = 9
cbatch = 15, cconcept = 13, length = 9
cbatch = 15, cconcept = 14, length = 9
cbatch = 15, cconcept = 15, length = 9
cbatch = 15, cconcept = 16, length = 9
cbatch = 16, cconcept = 1, length = 9
cbatch = 16, cconcept = 2, length = 9
cbatch = 16, cconcept = 3, length = 9
cbatch = 16, cconcept = 4, length = 9
cbatch = 16, cconcept = 5, length = 9
cbatch = 16, cconcept = 6, length = 9
cbatch = 16, cconcept = 7, length = 9
cbatch = 16, cconcept = 8, length = 9
cbatch = 16, cconcept = 9, length = 9
cbatch = 16, cconcept = 10, length = 9
cbat

cbatch = 30, cconcept = 1, length = 9
cbatch = 30, cconcept = 2, length = 9
cbatch = 30, cconcept = 3, length = 9
cbatch = 30, cconcept = 4, length = 9
cbatch = 30, cconcept = 5, length = 9
cbatch = 30, cconcept = 6, length = 9
cbatch = 30, cconcept = 7, length = 9
cbatch = 30, cconcept = 8, length = 9
cbatch = 30, cconcept = 9, length = 9
cbatch = 30, cconcept = 10, length = 9
cbatch = 30, cconcept = 11, length = 9
cbatch = 30, cconcept = 12, length = 9
cbatch = 30, cconcept = 13, length = 9
cbatch = 30, cconcept = 14, length = 9
cbatch = 30, cconcept = 15, length = 9
cbatch = 30, cconcept = 16, length = 9
cbatch = 31, cconcept = 1, length = 9
cbatch = 31, cconcept = 2, length = 9
cbatch = 31, cconcept = 3, length = 9
cbatch = 31, cconcept = 4, length = 9
cbatch = 31, cconcept = 5, length = 9
cbatch = 31, cconcept = 6, length = 9
cbatch = 31, cconcept = 7, length = 9
cbatch = 31, cconcept = 8, length = 9
cbatch = 31, cconcept = 9, length = 9
cbatch = 31, cconcept = 10, length = 9
cbat

cbatch = 45, cconcept = 1, length = 9
cbatch = 45, cconcept = 2, length = 9
cbatch = 45, cconcept = 3, length = 9
cbatch = 45, cconcept = 4, length = 9
cbatch = 45, cconcept = 5, length = 9
cbatch = 45, cconcept = 6, length = 9
cbatch = 45, cconcept = 7, length = 9
cbatch = 45, cconcept = 8, length = 9
cbatch = 45, cconcept = 9, length = 9
cbatch = 45, cconcept = 10, length = 9
cbatch = 45, cconcept = 11, length = 9
cbatch = 45, cconcept = 12, length = 9
cbatch = 45, cconcept = 13, length = 9
cbatch = 45, cconcept = 14, length = 9
cbatch = 45, cconcept = 15, length = 9
cbatch = 45, cconcept = 16, length = 9
cbatch = 46, cconcept = 1, length = 9
cbatch = 46, cconcept = 2, length = 9
cbatch = 46, cconcept = 3, length = 9
cbatch = 46, cconcept = 4, length = 9
cbatch = 46, cconcept = 5, length = 9
cbatch = 46, cconcept = 6, length = 9
cbatch = 46, cconcept = 7, length = 9
cbatch = 46, cconcept = 8, length = 9
cbatch = 46, cconcept = 9, length = 9
cbatch = 46, cconcept = 10, length = 9
cbat

cbatch = 59, cconcept = 1, length = 9
cbatch = 59, cconcept = 2, length = 9
cbatch = 59, cconcept = 3, length = 9
cbatch = 59, cconcept = 4, length = 9
cbatch = 59, cconcept = 5, length = 9
cbatch = 59, cconcept = 6, length = 9
cbatch = 59, cconcept = 7, length = 9
cbatch = 59, cconcept = 8, length = 9
cbatch = 59, cconcept = 9, length = 9
cbatch = 59, cconcept = 10, length = 9
cbatch = 59, cconcept = 11, length = 9
cbatch = 59, cconcept = 12, length = 9
cbatch = 59, cconcept = 13, length = 9
cbatch = 59, cconcept = 14, length = 9
cbatch = 59, cconcept = 15, length = 9
cbatch = 59, cconcept = 16, length = 9
cbatch = 60, cconcept = 1, length = 9
cbatch = 60, cconcept = 2, length = 9
cbatch = 60, cconcept = 3, length = 9
cbatch = 60, cconcept = 4, length = 9
cbatch = 60, cconcept = 5, length = 9
cbatch = 60, cconcept = 6, length = 9
cbatch = 60, cconcept = 7, length = 9
cbatch = 60, cconcept = 8, length = 9
cbatch = 60, cconcept = 9, length = 9
cbatch = 60, cconcept = 10, length = 9
cbat

...

cbatch = 2955, cconcept = 1, length = 9
cbatch = 2955, cconcept = 2, length = 9
cbatch = 2955, cconcept = 3, length = 9
cbatch = 2955, cconcept = 4, length = 9
cbatch = 2955, cconcept = 5, length = 9
cbatch = 2955, cconcept = 6, length = 9
cbatch = 2955, cconcept = 7, length = 9
cbatch = 2955, cconcept = 8, length = 9
cbatch = 2955, cconcept = 9, length = 9
cbatch = 2955, cconcept = 10, length = 9
cbatch = 2955, cconcept = 11, length = 9
cbatch = 2955, cconcept = 12, length = 9
cbatch = 2955, cconcept = 13, length = 9
cbatch = 2955, cconcept = 14, length = 9
cbatch = 2955, cconcept = 15, length = 9
cbatch = 2955, cconcept = 16, length = 9
cbatch = 2956, cconcept = 1, length = 9
cbatch = 2956, cconcept = 2, length = 9
cbatch = 2956, cconcept = 3, length = 9
cbatch = 2956, cconcept = 4, length = 9
cbatch = 2956, cconcept = 5, length = 9
cbatch = 2956, cconcept = 6, length = 9
cbatch = 2956, cconcept = 7, length = 9
cbatch = 2956, cconcept = 8, length = 9
cbatch = 2956, cconcept = 9, leng

cbatch = 2968, cconcept = 1, length = 9
cbatch = 2968, cconcept = 2, length = 9
cbatch = 2968, cconcept = 3, length = 9
cbatch = 2968, cconcept = 4, length = 9
cbatch = 2968, cconcept = 5, length = 9
cbatch = 2968, cconcept = 6, length = 9
cbatch = 2968, cconcept = 7, length = 9
cbatch = 2968, cconcept = 8, length = 9
cbatch = 2968, cconcept = 9, length = 9
cbatch = 2968, cconcept = 10, length = 9
cbatch = 2968, cconcept = 11, length = 9
cbatch = 2968, cconcept = 12, length = 9
cbatch = 2968, cconcept = 13, length = 9
cbatch = 2968, cconcept = 14, length = 9
cbatch = 2968, cconcept = 15, length = 9
cbatch = 2968, cconcept = 16, length = 9
cbatch = 2969, cconcept = 1, length = 9
cbatch = 2969, cconcept = 2, length = 9
cbatch = 2969, cconcept = 3, length = 9
cbatch = 2969, cconcept = 4, length = 9
cbatch = 2969, cconcept = 5, length = 9
cbatch = 2969, cconcept = 6, length = 9
cbatch = 2969, cconcept = 7, length = 9
cbatch = 2969, cconcept = 8, length = 9
cbatch = 2969, cconcept = 9, leng

cbatch = 2982, cconcept = 1, length = 9
cbatch = 2982, cconcept = 2, length = 9
cbatch = 2982, cconcept = 3, length = 9
cbatch = 2982, cconcept = 4, length = 9
cbatch = 2982, cconcept = 5, length = 9
cbatch = 2982, cconcept = 6, length = 9
cbatch = 2982, cconcept = 7, length = 9
cbatch = 2982, cconcept = 8, length = 9
cbatch = 2982, cconcept = 9, length = 9
cbatch = 2982, cconcept = 10, length = 9
cbatch = 2982, cconcept = 11, length = 9
cbatch = 2982, cconcept = 12, length = 9
cbatch = 2982, cconcept = 13, length = 9
cbatch = 2982, cconcept = 14, length = 9
cbatch = 2982, cconcept = 15, length = 9
cbatch = 2982, cconcept = 16, length = 9
cbatch = 2983, cconcept = 1, length = 9
cbatch = 2983, cconcept = 2, length = 9
cbatch = 2983, cconcept = 3, length = 9
cbatch = 2983, cconcept = 4, length = 9
cbatch = 2983, cconcept = 5, length = 9
cbatch = 2983, cconcept = 6, length = 9
cbatch = 2983, cconcept = 7, length = 9
cbatch = 2983, cconcept = 8, length = 9
cbatch = 2983, cconcept = 9, leng

cbatch = 2997, cconcept = 1, length = 9
cbatch = 2997, cconcept = 2, length = 9
cbatch = 2997, cconcept = 3, length = 9
cbatch = 2997, cconcept = 4, length = 9
cbatch = 2997, cconcept = 5, length = 9
cbatch = 2997, cconcept = 6, length = 9
cbatch = 2997, cconcept = 7, length = 9
cbatch = 2997, cconcept = 8, length = 9
cbatch = 2997, cconcept = 9, length = 9
cbatch = 2997, cconcept = 10, length = 9
cbatch = 2997, cconcept = 11, length = 9
cbatch = 2997, cconcept = 12, length = 9
cbatch = 2997, cconcept = 13, length = 9
cbatch = 2997, cconcept = 14, length = 9
cbatch = 2997, cconcept = 15, length = 9
cbatch = 2997, cconcept = 16, length = 9
cbatch = 2998, cconcept = 1, length = 9
cbatch = 2998, cconcept = 2, length = 9
cbatch = 2998, cconcept = 3, length = 9
cbatch = 2998, cconcept = 4, length = 9
cbatch = 2998, cconcept = 5, length = 9
cbatch = 2998, cconcept = 6, length = 9
cbatch = 2998, cconcept = 7, length = 9
cbatch = 2998, cconcept = 8, length = 9
cbatch = 2998, cconcept = 9, leng

cbatch = 3010, cconcept = 1, length = 9
cbatch = 3010, cconcept = 2, length = 9
cbatch = 3010, cconcept = 3, length = 9
cbatch = 3010, cconcept = 4, length = 9
cbatch = 3010, cconcept = 5, length = 9
cbatch = 3010, cconcept = 6, length = 9
cbatch = 3010, cconcept = 7, length = 9
cbatch = 3010, cconcept = 8, length = 9
cbatch = 3010, cconcept = 9, length = 9
cbatch = 3010, cconcept = 10, length = 9
cbatch = 3010, cconcept = 11, length = 9
cbatch = 3010, cconcept = 12, length = 9
cbatch = 3010, cconcept = 13, length = 9
cbatch = 3010, cconcept = 14, length = 9
cbatch = 3010, cconcept = 15, length = 9
cbatch = 3010, cconcept = 16, length = 9
cbatch = 3011, cconcept = 1, length = 9
cbatch = 3011, cconcept = 2, length = 9
cbatch = 3011, cconcept = 3, length = 9
cbatch = 3011, cconcept = 4, length = 9
cbatch = 3011, cconcept = 5, length = 9
cbatch = 3011, cconcept = 6, length = 9
cbatch = 3011, cconcept = 7, length = 9
cbatch = 3011, cconcept = 8, length = 9
cbatch = 3011, cconcept = 9, leng

In [55]:
concept_map = next(batch)
print(len(concept_map))


16


In [56]:
img_index = concept_map[0]['i']
scalars, pixels = [], []
print(img_index)

32


In [ ]:
for cat in data.category_names():
    # get corresponding 
    label_group = concept_map[cat]
    shape = np.shape(label_group)
    if len(shape) % 2 == 0:
        label_group = [label_group]
    if len(shape) < 2:
        scalars += label_group
    else:
        pixels.append(label_group)

    for scalar in scalars:
        tally_labels[scalar] += concept_map['sh'] * concept_map['sw']
        if pixels:
            pixels = np.concatenate(pixels)
            tally_label = np.bincount(pixels.ravel())
            if len(tally_label) > 0:
                tally_label[0] = 0
                tally_labels[:len(tally_label)] += tally_label


In [47]:
for layer_id, layer in enumerate(FEATURE_NAMES):
    print(layer_id, layer)
    tally_result = fo.tally(features[layer_id],thresholds,savepath="tally.csv")

0 layer4


/software/Anaconda3-5.0.1-el7-x86_64/envs/DL_GPU/lib/python3.6/site-packages/ipykernel/__main__.py:128: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/software/Anaconda3-5.0.1-el7-x86_64/envs/DL_GPU/lib/python3.6/site-packages/ipykernel/__main__.py:128: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/software/Anaconda3-5.0.1-el7-x86_64/envs/DL_GPU/lib/python3.6/site-packages/ipykernel/__main__.py:128: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/software/Anaconda3-5.0.1-el7-x86_64/envs/DL_GPU/lib/python3.6/site-packages/ipykernel/__main__.py:128: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/sof

FileNotFoundError: [Errno 2] No such file or directory: '../../../project/rcc/deep_learning_hack/netdissect/broden1_224/images/a'

In [151]:
d.category_names()

['color', 'object', 'part', 'scene', 'texture']

In [152]:
d.label

[{'number': 0,
  'name': '',
  'category': {},
  'frequency': 0,
  'coverage': 0.0,
  'syns': []},
 {'number': 1,
  'name': 'black-c',
  'category': {'color': 62358},
  'frequency': 62358,
  'coverage': 11135.320474,
  'syns': ['']},
 {'number': 2,
  'name': 'grey-c',
  'category': {'color': 62310},
  'frequency': 62310,
  'coverage': 12712.843129,
  'syns': ['']},
 {'number': 3,
  'name': 'white-c',
  'category': {'color': 62054},
  'frequency': 62054,
  'coverage': 5778.038204,
  'syns': ['']},
 {'number': 4,
  'name': 'brown-c',
  'category': {'color': 61583},
  'frequency': 61583,
  'coverage': 14549.652454,
  'syns': ['']},
 {'number': 5,
  'name': 'green-c',
  'category': {'color': 61508},
  'frequency': 61508,
  'coverage': 5049.989735,
  'syns': ['']},
 {'number': 6,
  'name': 'pink-c',
  'category': {'color': 60755},
  'frequency': 60755,
  'coverage': 1478.518582,
  'syns': ['']},
 {'number': 7,
  'name': 'purple-c',
  'category': {'color': 59516},
  'frequency': 59516,
  'co

## data_loader.py

In [6]:
def load_csv(filename, readfields=None):
    def convert(value):
        if re.match(r'^-?\d+$', value): # ok regular expression stuffs
            try:
                return int(value)
            except:
                pass
        if re.match(r'^-?[\.\d]+(?:e[+=]\d+)$', value):
            try:
                return float(value)
            except:
                pass
        return value
    
    with open(filename) as f:
        reader = csv.DictReader(f)
        result = [{k: convert(v) for k, v in row.items()} for row in reader]
        if readfields is not None:
            readfields.extend(reader.fieldnames)
    return result